In [1]:
library(Matching)
library(tidyverse)

Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks MASS::select()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps_subgroup_vaso.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7339584,1.362475
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3889232,2.571202
214106,123152,74869,MICU,2150-09-05 17:50:32,2150-09-12 21:40:59,1,74.13371,1,2150-09-07,⋯,1,1,1,1,1,1,1,1,0.7721262,1.295125
254478,128652,31,MICU,2108-08-22 23:28:42,2108-08-30 21:59:20,1,72.26709,1,NA,⋯,1,1,1,0,1,1,0,1,0.1951075,1.242402
264446,165520,10013,MICU,2125-10-04 23:38:00,2125-10-07 15:13:52,1,87.08742,1,NA,⋯,1,1,1,0,1,0,0,1,0.4902299,1.961669
218143,122936,10370,SICU,2145-01-15 03:36:30,2145-01-20 19:00:30,1,45.06368,1,2145-01-14,⋯,1,1,1,0,1,1,1,0,0.8799575,1.136419


In [4]:
set.seed(3485)

In [5]:
ps_matches <- Match(Y = full_data$mort_28_day_int, Tr = full_data$echo_int,
                    X = full_data$ps, M = 1, estimand = "ATT", caliper = 0.01,
                    exact = FALSE, replace = FALSE, ties = FALSE)

Warning message in Match(Y = full_data$mort_28_day_int, Tr = full_data$echo_int, :
“replace==FALSE, but there are more (weighted) treated obs than control obs.  Some treated obs will not be matched.  You may want to estimate ATC instead.”

In [6]:
ps_matches_df <-
full_data[ps_matches$index.treated, "icustay_id", drop = FALSE] %>%
mutate(match = full_data[ps_matches$index.control, "icustay_id"]) %>%
rename(trtd = icustay_id, ctrl = match) %>%
mutate(match = 1:n()) %>%
gather("group", "icustay_id", trtd, ctrl) %>%
select(icustay_id, group, match) %>%
arrange(group, match)

head(ps_matches_df)

icustay_id,group,match
228283,ctrl,1
249855,ctrl,2
280310,ctrl,3
237390,ctrl,4
293409,ctrl,5
245467,ctrl,6


In [7]:
# saveRDS(ps_matches, file = file.path(data_dir, "ps_matches.rds"))
data.table::fwrite(ps_matches_df, file.path(data_dir, "ps_matches_df_subgroup_vaso.csv"))

In [8]:
ids <- full_data[unlist(ps_matches[c("index.control", "index.treated")]), "icustay_id"]
length(ids) / 2
head(ids)

[1] 412

[1] 228283 249855 280310 237390 293409 245467

In [9]:
data.table::fwrite(data.frame(icustay_id = ids), file = file.path(data_dir, "ps_icustay_id"))

In [10]:
summary(ps_matches)


Estimate...  -0.10437 
SE.........  0.033501 
T-stat.....  -3.1154 
p.val......  0.0018373 

Original number of observations..............  2067 
Original number of treated obs...............  1228 
Matched number of observations...............  412 
Matched number of observations  (unweighted).  412 

Number of obs dropped by 'exact' or 'caliper'  816 



In [11]:
outcome <- data.frame(echo_pt = full_data$mort_28_day_int[ps_matches$index.treated],
                      match_pt = full_data$mort_28_day_int[ps_matches$index.control])

In [12]:
head(outcome)

echo_pt,match_pt
0,0
0,0
1,1
0,1
1,0
1,1


In [13]:
tab <- table(outcome$echo_pt, outcome$match_pt, dnn = c("Echo", "Non-Echo"))
tab
tab[2, 1] / tab[1, 2]
paste("95% Confint",
      round(exp(c(log(tab[2, 1] / tab[1, 2]) - qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]),
                  log(tab[2, 1] / tab[1, 2]) + qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]))), 7))
mcnemar.test(tab) # for 1-1 pairs

    Non-Echo
Echo   0   1
   0 156 119
   1  76  61

[1] 0.6386555

[1] "95% Confint 0.4789369" "95% Confint 0.8516378"


	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 9.0462, df = 1, p-value = 0.002632


In [14]:
(tab[2, 2] + tab[1, 2]) / sum(tab)
(tab[2, 2] + tab[2, 1]) / sum(tab)

[1] 0.4368932

[1] 0.3325243

In [15]:
mortality <- data.frame(echo = c(0, 1),
                        mortality = c((tab[2, 2] + tab[1, 2]) / sum(tab),
                                      (tab[2, 2] + tab[2, 1]) / sum(tab)))
mortality

echo,mortality
0,0.4368932
1,0.3325243


In [16]:
mort_fmt <- mortality %>%
mutate(echo = case_when(
    echo == 0 ~ "Non_echo",
    echo == 1 ~ "Echo"
)) %>%
select(echo, mortality) %>%
spread(echo, mortality) %>%
mutate(Method = "PS Match") %>%
select(Method, Echo, Non_echo)
mort_fmt

Method,Echo,Non_echo
PS Match,0.3325243,0.4368932


In [17]:
data.table::fwrite(mort_fmt, file.path(data_dir, "mort_ps_subgroup_vaso.csv"))